In [11]:
import zipfile
from pathlib import Path
import partitura
import pandas as pd
import xml.etree.ElementTree as ET
from spacy import displacy
import numpy as np

In [5]:
#unzip all files in zips folder

for zip_file in Path("zips").iterdir():
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(Path("unzipped"))

In [7]:
# check if ts trees are in unzipped folder
for folder in Path("unzipped").iterdir():
    if not any([file.name.startswith("TS") for file in folder.iterdir()]):
        print("not a TS tree in ", folder)

In [ ]:
# check if there are scores everywhere
for folder in Path("unzipped").iterdir():
    msc_file = [file for file in folder.iterdir() if file.name.startswith("MSC")]
    numbered_file = [file for file in folder.iterdir() if file.name.startswith(folder.name[:2])]
    assert(len(msc_file)+len(numbered_file) == 1)
    score_file = msc_file[0] if len(msc_file) == 1 else numbered_file[0]
    try:
        score = partitura.load_musicxml(str(score_file))
    except Exception as e:
        print("error loading ", score_file, e)

In [2]:
# build a pandas dataframe with all the scores paths
def get_score_path(folder):
    msc_file = [file for file in folder.iterdir() if file.name.startswith("MSC")]
    numbered_file = [file for file in folder.iterdir() if file.name.startswith(folder.name[:2])]
    assert(len(msc_file)+len(numbered_file) == 1)
    score_file = msc_file[0] if len(msc_file) == 1 else numbered_file[0]
    return str(score_file)

def get_ts_path(folder):
    ts_file = [file for file in folder.iterdir() if file.name.startswith("TS")]
    assert(len(ts_file) == 1)
    return str(ts_file[0])

list_of_tuples = []

for folder in Path("data").iterdir():
    list_of_tuples.append((get_score_path(folder), get_ts_path(folder)))

df = pd.DataFrame(list_of_tuples, columns = ['score', 'ts'])

In [3]:
df

,score,ts
0,data\01\01_Waltz in E flat Grande Valse Brilla...,data\01\TS-01.xml
1,data\02\02_Moments Musicaux.xml,data\02\TS-02.xml
2,data\03\03_Bagatelle 'Fur Elise' WoO.59.xml,data\03\TS-03.xml
3,data\04\04_The Preludes Op.28 No.15.xml,data\04\TS-04.xml
4,data\05\05_Turkish March.xml,data\05\TS-05.xml
...,...,...
328,data\95\95_12 Variationen uber ein franzosisch...,data\95\TS-95.xml
329,data\96\96_Lieder ohne Worte Heft 5 Op.62-6 Fr...,data\96\TS-96.xml
330,data\97\97_Les Patineurs Op.183.xml,data\97\TS-97.xml
331,data\98\98_Sonate fur Klavier Nr.8 c moll Path...,data\98\TS-98.xml


# Parse tree annotations from the xml file

In [12]:
# parse the xml file with annotations in a list of dependencies
tree = ET.parse(r'data\07\TS-07.xml')
root = tree.getroot()

In [45]:
# def iterative_parse(xml_elem):
#     primary_children = xml_elem.find("ts").find("primary")
#     secondary_children = xml_elem.find("ts").find("secondary")
#     # recursion ending condition
#     if primary_children is None:
#         assert secondary_children is None
#         return xml_elem.find("ts").find("head").find("chord").find("note").attrib["id"]
#     else:
#         assert secondary_children is not None
#         return [iterative_parse(primary_children), iterative_parse(secondary_children)]


# iterative_parse(root)

In [13]:
def iterative_parse(xml_elem):
    primary_children = xml_elem.find("ts").find("primary")
    secondary_children = xml_elem.find("ts").find("secondary")
    if primary_children is None: # recursion ending condition
        assert secondary_children is None
        return [], xml_elem.find("ts").find("head").find("chord").find("note").attrib["id"]
    else: # recursive call
        assert secondary_children is not None
        out_list = [] # dependency list
        iterative_result_primary = iterative_parse(primary_children)
        iterative_result_secondary = iterative_parse(secondary_children)
        # merge the dependencies lists computed deeper
        out_list.extend(iterative_result_primary[0])
        out_list.extend(iterative_result_secondary[0])
        # append the dependency for the current node
        out_list.append((iterative_result_primary[1], iterative_result_secondary[1]))
        # return the dependency list, and the id of the current node, i.e., the primary
        return out_list, iterative_parse(primary_children)[1]


arcs = iterative_parse(root)[0]
arcs

[('P1-9-5', 'P1-9-4'),
 ('P1-9-7', 'P1-9-6'),
 ('P1-9-5', 'P1-9-7'),
 ('P1-9-9', 'P1-9-8'),
 ('P1-9-5', 'P1-9-9'),
 ('P1-9-10', 'P1-9-5'),
 ('P1-9-1', 'P1-9-2'),
 ('P1-9-1', 'P1-9-3'),
 ('P1-9-10', 'P1-9-1'),
 ('P1-8-5', 'P1-8-4'),
 ('P1-8-6', 'P1-8-5'),
 ('P1-8-2', 'P1-8-3'),
 ('P1-8-1', 'P1-7-15'),
 ('P1-8-2', 'P1-8-1'),
 ('P1-8-6', 'P1-8-2'),
 ('P1-9-10', 'P1-8-6'),
 ('P1-6-1', 'P1-5-12'),
 ('P1-6-9', 'P1-6-8'),
 ('P1-6-7', 'P1-6-6'),
 ('P1-6-2', 'P1-6-3'),
 ('P1-6-4', 'P1-6-5'),
 ('P1-6-2', 'P1-6-4'),
 ('P1-6-7', 'P1-6-2'),
 ('P1-6-9', 'P1-6-7'),
 ('P1-6-1', 'P1-6-9'),
 ('P1-6-15', 'P1-6-14'),
 ('P1-6-12', 'P1-6-13'),
 ('P1-6-11', 'P1-6-12'),
 ('P1-6-15', 'P1-6-11'),
 ('P1-7-1', 'P1-6-15'),
 ('P1-6-1', 'P1-7-1'),
 ('P1-7-13', 'P1-7-12'),
 ('P1-7-14', 'P1-7-13'),
 ('P1-7-3', 'P1-7-2'),
 ('P1-7-5', 'P1-7-4'),
 ('P1-7-3', 'P1-7-5'),
 ('P1-7-11', 'P1-7-10'),
 ('P1-7-7', 'P1-7-6'),
 ('P1-7-9', 'P1-7-8'),
 ('P1-7-7', 'P1-7-9'),
 ('P1-7-11', 'P1-7-7'),
 ('P1-7-3', 'P1-7-11'),
 ('P1-7-14',

In [53]:
import numpy as np


In [14]:
note_ids = list(np.unique([e.attrib['id'] for e in root.findall('''.//note''')]))

spacy_words = [{"text": word, "tag": ""} for word in note_ids]
spacy_arcs = []
for (start_id, end_id) in arcs:
    start_ix = note_ids.index(start_id)
    end_ix = note_ids.index(end_id)
    if start_ix < end_ix:
        spacy_arcs.append({"start": start_ix, "end": end_ix, "label": "", "dir": "right"})
    else:
        spacy_arcs.append({"start": end_ix, "end": start_ix, "label": "", "dir": "left"})


spacy_dict = {"words": spacy_words, "arcs": spacy_arcs}
displacy.render(spacy_dict, style='dep', jupyter=True, manual=True)

In [18]:
import spacy
from spacy import displacy

# Load the language model
nlp = spacy.load("en_core_web_sm")

sentence = 'Deemed universities charge huge fees'

# nlp function returns an object with individual token information, 
# linguistic features and relationships
doc = nlp(sentence)

print ("{:<15} | {:<8} | {:<15} | {:<20}".format('Token','Relation','Head', 'Children'))
print ("-" * 70)

for token in doc:
  # Print the token, dependency nature, head and all dependents of the token
  print ("{:<15} | {:<8} | {:<15} | {:<20}"
         .format(str(token.text), str(token.dep_), str(token.head.text), str([child for child in token.children])))
  
# Use displayCy to visualize the dependency 
displacy.render(doc, style='dep', jupyter=True, options={'distance': 120})

Token           | Relation | Head            | Children            
----------------------------------------------------------------------
Deemed          | amod     | universities    | []                  
universities    | nsubj    | charge          | [Deemed]            
charge          | ROOT     | charge          | [universities, fees]
huge            | amod     | fees            | []                  
fees            | dobj     | charge          | [huge]              


In [22]:
l = []
for token in doc:
    l.append(token)
type(doc)

spacy.tokens.doc.Doc

In [31]:
obj = {
    "words": [
        {"text": "This", "tag": ""},
        {"text": "is", "tag": ""},
        {"text": "a", "tag": ""},
        {"text": "sentence", "tag": ""}
    ],
    "arcs": [
        {"start": 0, "end": 1, "label": "", "dir": "right"},
        {"start": 2, "end": 3, "label": "", "dir": "left"},
        {"start": 1, "end": 3, "label": "", "dir": "right"}
    ]
}

displacy.render(obj, style='dep', jupyter=True, options={'distance': 120}, manual=True)

# Parse note features from score file

In [5]:
score_path = df.iloc[6]["score"]
score_path

'data\\07\\07_Nocturne.xml'

In [6]:
score = partitura.load_musicxml(score_path)
len(score.parts[0].rests)

1

In [25]:
extended_score_note_array = partitura.utils.music.ensure_notearray(
    score,
    include_metrical_position=True, # adds 3 fields: is_downbeat, rel_onset_div, tot_measure_div
    include_time_signature=True, # adds 2 fields: time_signature_numerator, time_signature_denominator
)

In [33]:
np.char.array(extended_score_note_array["ts_beats"]) + np.char.array(extended_score_note_array["ts_beat_type"])

chararray([b'34', b'34', b'34', b'34', b'34', b'34', b'34', b'34', b'34',
           b'34', b'34', b'34', b'34', b'34', b'34', b'34', b'34', b'34',
           b'34', b'34', b'34', b'34', b'34', b'34', b'34', b'34', b'34',
           b'34'], dtype='|S2')

In [35]:
scores = [partitura.load_musicxml(score_path) for score_path in df["score"]]  
    

c:\Users\fosca\anaconda3\envs\mir_research\lib\site-packages\partitura\directions.py:533: UserWarning: error parsing "sentito" (UnexpectedCharacters)
  warnings.warn('error parsing "{}" ({})'.format(string, type(e).__name__))
c:\Users\fosca\anaconda3\envs\mir_research\lib\site-packages\partitura\directions.py:533: UserWarning: error parsing "Largamente" (UnexpectedCharacters)
  warnings.warn('error parsing "{}" ({})'.format(string, type(e).__name__))
c:\Users\fosca\anaconda3\envs\mir_research\lib\site-packages\partitura\directions.py:533: UserWarning: error parsing "cresc.  -   -   -  " (UnexpectedCharacters)
  warnings.warn('error parsing "{}" ({})'.format(string, type(e).__name__))
c:\Users\fosca\anaconda3\envs\mir_research\lib\site-packages\partitura\directions.py:533: UserWarning: error parsing "cre    -     -     -     scen     -    -    -     do" (UnexpectedCharacters)
  warnings.warn('error parsing "{}" ({})'.format(string, type(e).__name__))
c:\Users\fosca\anaconda3\envs\mir_re

In [64]:
all_time_signatures = []

for i,score in enumerate(scores):
    na = partitura.utils.music.ensure_notearray(
        score,
        include_metrical_position=True,
        include_time_signature=True,        
    )
    # if len(score.parts[0].rests) > 0:
    #     print(score_path, len(score.parts[0].rests))
    time_signatures = np.char.array(na["ts_beats"].astype(str)) + np.char.array(["/"]*na.shape[0])+ np.char.array(na["ts_beat_type"].astype(str))
    all_time_signatures.append(np.unique(time_signatures)[-1])
    if np.unique(time_signatures).shape[0] > 1:
        print(np.unique(time_signatures), df["score"][i], np.unique(time_signatures)[-1] )

['1/8' '2/8' '3/8'] data\03\03_Bagatelle 'Fur Elise' WoO.59.xml 3/8
['1/8' '5/8' '6/8'] data\06\06_Blumenlied Op.39.xml 6/8
['1/8' '11/8' '12/8'] data\07\07_Nocturne.xml 12/8
['1/4' '3/4' '4/4'] data\09\09_String Quartet in F major Op.3 No.5 Serenade.xml 4/4
['1/4' '2/4' '3/4'] data\10\10_Wiegenlied.xml 3/4
['1/4' '4/4'] data\11\11_Solvejgs Lied.xml 4/4
['2/4' '3/4'] data\115\MSC-115.xml 3/4
['1/4' '2/4' '3/4'] data\12\12_Anitras Dans.xml 3/4
['1/4' '3/4' '4/4'] data\13\13_Traumerei.xml 4/4
['1/4' '2/4' '3/4'] data\14\14_Menuett No.2 in G maj.xml 3/4
['1/4' '3/4' '4/4'] data\16\16_William Tell Overture.xml 4/4
['1/4' '2/4' '3/4'] data\19\19_Tannhauser Overture.xml 3/4
['1/8' '2/8' '3/8'] data\20\20_La Traviata Brindisi.xml 3/8
['1/8' '5/8' '6/8'] data\21\21_Plaisir d'Amour.xml 6/8
['2/4' '4/4'] data\25\25_L'arlesienne Suite No.2 Farandole.xml 4/4
['1/4' '2/2' '3/4'] data\37\37_Sonate fur Klavier Nr.48 C dur Op.30-1 Mov.1.xml 3/4
['1/4' '2/4' '3/4'] data\49\49_The Planets Op.32 Jupiter,

In [66]:
np.unique(all_time_signatures)
# for i, ts in enumerate(all_time_signatures):
#     print(ts, df["score"][i])

array(['12/8', '2/2', '2/4', '3/2', '3/4', '3/8', '4/4', '6/4', '6/8',
       '9/8'], dtype='<U4')

In [62]:
np.char.array([","]*na.shape[0]) + np.char.array(na["ts_beat_type"].astype(str))

chararray([',4', ',4', ',4', ',4', ',4', ',4', ',4', ',4', ',4', ',4',
           ',4', ',4', ',4', ',4', ',4', ',4', ',4', ',4', ',4', ',4',
           ',4', ',4', ',4', ',4', ',4', ',4', ',4', ',4'], dtype='<U12')

In [74]:
score_path = df.iloc[6]["score"]
score = partitura.load_musicxml(score_path)

In [110]:
na = partitura.utils.music.ensure_notearray(
        score,
        include_metrical_position=True,
        include_time_signature=True,    
    )

na, real_ts,real_measure_duration = correct_metrical_information(na)

na
# na

array([( 0. , 1. ,  0.  , 0.5 ,   0,  2, 70, 1, 'None', 12, 8, 12, 1, 22, 24, 4),
       ( 1. , 4. ,  0.5 , 2.  ,   2,  8, 79, 1, 'None', 12, 8, 12, 1,  0, 24, 4),
       ( 5. , 1. ,  2.5 , 0.5 ,  10,  2, 77, 1, 'None', 12, 8, 12, 0,  8, 24, 4),
       ( 6. , 1. ,  3.  , 0.5 ,  12,  2, 79, 1, 'None', 12, 8, 12, 0, 10, 24, 4),
       ( 7. , 3. ,  3.5 , 1.5 ,  14,  6, 77, 1, 'None', 12, 8, 12, 0, 12, 24, 4),
       (10. , 2. ,  5.  , 1.  ,  20,  4, 75, 1, 'None', 12, 8, 12, 0, 18, 24, 4),
       (12. , 1. ,  6.  , 0.5 ,  24,  2, 70, 1, 'None', 12, 8, 12, 0, 22, 24, 4),
       (13. , 2. ,  6.5 , 1.  ,  26,  4, 79, 1, 'None', 12, 8, 12, 1,  0, 24, 4),
       (15. , 1. ,  7.5 , 0.5 ,  30,  2, 72, 1, 'None', 12, 8, 12, 0,  4, 24, 4),
       (16. , 2. ,  8.  , 1.  ,  32,  4, 84, 1, 'None', 12, 8, 12, 0,  6, 24, 4),
       (18. , 1. ,  9.  , 0.5 ,  36,  2, 79, 1, 'None', 12, 8, 12, 0, 10, 24, 4),
       (19. , 3. ,  9.5 , 1.5 ,  38,  6, 82, 1, 'None', 12, 8, 12, 0, 12, 24, 4),
       (22. , 2.

In [109]:
def correct_metrical_information(na):
    time_signatures = np.char.array(na["ts_beats"].astype(str)) + np.char.array(["/"]*na.shape[0])+ np.char.array(na["ts_beat_type"].astype(str))
    # get real time signature and measure duration, discarding pickup and ending measure ts
    real_ts = np.unique(time_signatures)[-1]
    real_measure_duration = na[time_signatures == real_ts][0]["tot_measure_div"]
    # find pickup notes
    pickup_note_indices = np.where((na["tot_measure_div"]!= real_measure_duration) * (na["onset_div"] < real_measure_duration ))[0]
    # set the pickup note to the correct metrical position
    na["rel_onset_div"][pickup_note_indices] = na["onset_div"][pickup_note_indices] + real_measure_duration - na["tot_measure_div"][pickup_note_indices]
    # set the measure duration to correct value
    na["tot_measure_div"] = real_measure_duration
    # set the correct time signature
    na["ts_beats"] = real_ts.split("/")[0]
    na["ts_mus_beats"] = real_ts.split("/")[0]
    na["ts_beat_type"] = real_ts.split("/")[1]
    return na, real_ts, real_measure_duration

def get_note_features(na):
    pitch = na["pitch"]
    metrical =  na["is_downbeat"] #TODO: add more metrical info
    duration = na["duration_div"]/na["tot_measure_div"]
    # TODO: consider rests as lag from previous note
    return np.concatenate((pitch, metrical, duration), axis=1)


In [105]:
na

array([( 0. , 1. ,  0.  , 0.5 ,   0,  2, 70, 1, 'None',  1, 8,  1, 1, 22,  2, 4),
       ( 1. , 4. ,  0.5 , 2.  ,   2,  8, 79, 1, 'None', 12, 8,  4, 1,  0, 24, 4),
       ( 5. , 1. ,  2.5 , 0.5 ,  10,  2, 77, 1, 'None', 12, 8,  4, 0,  8, 24, 4),
       ( 6. , 1. ,  3.  , 0.5 ,  12,  2, 79, 1, 'None', 12, 8,  4, 0, 10, 24, 4),
       ( 7. , 3. ,  3.5 , 1.5 ,  14,  6, 77, 1, 'None', 12, 8,  4, 0, 12, 24, 4),
       (10. , 2. ,  5.  , 1.  ,  20,  4, 75, 1, 'None', 12, 8,  4, 0, 18, 24, 4),
       (12. , 1. ,  6.  , 0.5 ,  24,  2, 70, 1, 'None', 12, 8,  4, 0, 22, 24, 4),
       (13. , 2. ,  6.5 , 1.  ,  26,  4, 79, 1, 'None', 12, 8,  4, 1,  0, 24, 4),
       (15. , 1. ,  7.5 , 0.5 ,  30,  2, 72, 1, 'None', 12, 8,  4, 0,  4, 24, 4),
       (16. , 2. ,  8.  , 1.  ,  32,  4, 84, 1, 'None', 12, 8,  4, 0,  6, 24, 4),
       (18. , 1. ,  9.  , 0.5 ,  36,  2, 79, 1, 'None', 12, 8,  4, 0, 10, 24, 4),
       (19. , 3. ,  9.5 , 1.5 ,  38,  6, 82, 1, 'None', 12, 8,  4, 0, 12, 24, 4),
       (22. , 2.

# Test data loader

In [1]:
from musicparser.data_loading import TSDataset
from pathlib import Path

c:\Users\fosca\anaconda3\envs\mir_research\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset = TSDataset(Path("data"))

UFuncTypeError: ufunc 'subtract' did not contain a loop with signature matching types (dtype('<U1'), dtype('<U1')) -> None

In [20]:
for e in dataset:  
    print(e)
    

(array([[70.        ,  1.        ,  0.33333333],
       [74.        ,  0.        ,  0.16666667],
       [75.        ,  0.        ,  0.16666667],
       [77.        ,  0.        ,  0.33333333],
       [70.        ,  1.        ,  0.33333333],
       [75.        ,  0.        ,  0.16666667],
       [77.        ,  0.        ,  0.16666667],
       [79.        ,  0.        ,  0.33333333],
       [70.        ,  1.        ,  0.33333333],
       [77.        ,  0.        ,  0.16666667],
       [79.        ,  0.        ,  0.16666667],
       [80.        ,  0.        ,  0.33333333],
       [82.        ,  1.        ,  0.66666667],
       [82.        ,  0.        ,  0.16666667],
       [82.        ,  0.        ,  0.16666667],
       [82.        ,  1.        ,  0.33333333],
       [84.        ,  0.        ,  0.16666667],
       [82.        ,  0.        ,  0.16666667],
       [80.        ,  0.        ,  0.33333333],
       [80.        ,  1.        ,  0.33333333],
       [82.        ,  0.        ,  0.16

# Connect xml ids with partitura ids

In [1]:
from musicparser.data_loading import ts_xml_to_dependency_tree, get_data_df, ts_xml_to_dependency_tree, gttm_style_to_id_dependency_ts
import partitura as pt
from pathlib import Path
import partitura as pt
import numpy as np
from spacy import displacy

df = get_data_df(Path("data"))
row = df.iloc[6]

ts_tree = ts_xml_to_dependency_tree(row["ts"])
score = pt.load_musicxml(row["score"],force_note_ids=True)
na = pt.utils.music.ensure_notearray(
        score,
        include_metrical_position=True,
        include_time_signature=True,    
    )

# ra = score.parts[0].rest_array()
# ra_fields = list(ra.dtype.names)
# # join rest array and note array (only the common fields), and reorder
# nra = np.hstack([na[ra_fields],ra])
# nra.sort(order="onset_div")
# nra

c:\Users\fosca\anaconda3\envs\mir_research\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
na_untied = pt.utils.music.note_array_from_note_list(score.parts[0].notes)
ra_untied = pt.utils.music.rest_array_from_rest_list(score.parts[0].rests)
ra__untied_fields = list(ra_untied.dtype.names)
nra_untied = np.hstack([na_untied[ra__untied_fields],ra_untied])
nra_untied.sort(order="onset_div")
# nra_untied

In [3]:
# note_measures = score.parts[0].measure_number_map(na["onset_div"])
# note_measures

In [4]:
# np.where(note_measures == 2)[0][3]

In [5]:
# def gttm_id_to_pt_id(gttm_id, measure_mapping):
#     """Translate the gttm-style ids, e.g., 'P1-3-1', to indices in partitura note array.

#     Args:
#         gttm_id (string): gttm-style id, e.g., 'P1-3-1'
#         measure_mapping (list[int]): a list with the measure number for each note in the partitura note array

#     Returns:
#         int: index of the note in the partitura note array
#     """
#     measure_number = int(gttm_id.split("-")[1])
#     note_number = int(gttm_id.split("-")[2])
#     return np.where(measure_mapping == measure_number)[0][int(note_number)-1]

# gttm_id_to_pt_id("P1-3-1", note_measures)

In [8]:
gttm_ts = ts_xml_to_dependency_tree(row["ts"])
m_map = score.parts[0].measure_number_map(nra_untied["onset_div"])
na_ts = gttm_style_to_id_dependency_ts(gttm_ts, m_map, nra_untied, na)
# na_ts

In [16]:
# note_ids = list(np.unique([e.attrib['id'] for e in root.findall('''.//note''')]))

note_ids = [str(e) for e in range(len(na))]
print(note_ids)

spacy_words = [{"text": word, "tag": ""} for word in note_ids]
spacy_arcs = []
for (start_id, end_id) in na_ts:
    start_ix = note_ids.index(str(start_id))
    end_ix = note_ids.index(str(end_id))
    if start_ix < end_ix:
        spacy_arcs.append({"start": start_ix, "end": end_ix, "label": "", "dir": "right"})
    else:
        spacy_arcs.append({"start": end_ix, "end": start_ix, "label": "", "dir": "left"})


spacy_dict = {"words": spacy_words, "arcs": spacy_arcs}
displacy.render(spacy_dict, style='dep', jupyter=True, manual=True)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74']


In [23]:
for score, ts in df.values:
    print(score,"LLLL",ts)

data\01\01_Waltz in E flat Grande Valse Brillante Op.18.xml LLLL data\01\TS-01.xml
data\02\02_Moments Musicaux.xml LLLL data\02\TS-02.xml
data\03\03_Bagatelle 'Fur Elise' WoO.59.xml LLLL data\03\TS-03.xml
data\04\04_The Preludes Op.28 No.15.xml LLLL data\04\TS-04.xml
data\05\05_Turkish March.xml LLLL data\05\TS-05.xml
data\06\06_Blumenlied Op.39.xml LLLL data\06\TS-06.xml
data\07\07_Nocturne.xml LLLL data\07\TS-07.xml
data\08\08_Spinnerlied Op.14 No.4.xml LLLL data\08\TS-08.xml
data\09\09_String Quartet in F major Op.3 No.5 Serenade.xml LLLL data\09\TS-09.xml
data\10\10_Wiegenlied.xml LLLL data\10\TS-10.xml
data\100\100_Sonate fur Klavier Nr.11 A dur K.331 K6.300i Mov.1.xml LLLL data\100\TS-100.xml
data\101\MSC-101.xml LLLL data\101\TS-101.xml
data\102\MSC-102.xml LLLL data\102\TS-102.xml
data\103\MSC-103.xml LLLL data\103\TS-103.xml
data\104\MSC-104.xml LLLL data\104\TS-104.xml
data\105\MSC-105.xml LLLL data\105\TS-105.xml
data\106\MSC-106.xml LLLL data\106\TS-106.xml
data\107\MSC-107